#### Import

In [2]:
import numpy as np
import numba as nb
from scipy.stats import gaussian_kde

#### Functions

In [3]:
@nb.njit(nb.float64(nb.float64))
def get_phi(x):
    return np.exp(-x**2 / 2) / np.sqrt(2*np.pi)


@nb.njit(nb.float64(nb.float64[:]))
def silverman_bandwidth(data):
    if len(data) < 2:
        raise ValueError("Need at least 2 data points")
    
    std = np.std(data)
    IQR = np.quantile(data, .75) - np.quantile(data, .25)
    A = min(std, IQR / 1.34)
    return 0.9 * A * len(data)**(-0.2)


@nb.njit(nb.float64(nb.float64[:], nb.float64))
def custom_kde(data, point):
    n = len(data)
    h = silverman_bandwidth(data)
    result = 0
    for i in range(n):
        result += get_phi((point - data[i]) / h) / (n * h)
    return result

#### Test

In [4]:
n = 10
np.random.seed(1000002)
data = np.random.randn(n)
point = .7

In [8]:
bw = silverman_bandwidth(data)

gaussian_kde(data, bw_method=bw/np.std(data, ddof=1))(point)

array([0.47349868])

In [9]:
custom_kde(data, point)

0.473498677174841